# Lungs assisted with mechanical ventilation: Pressure-controlled ventilation (PCV) mode.

In the following notebook we describe the methodology to simulate a PCV example using the approach proposal in Avilés & Hurtado (2022).

We begin by importing the necessary modules.

In [ ]:
# Import and install
try:
    import dolfin
    print("oui")
except ImportError:
    !wget "https://fem-on-colab.github.io/releases/fenics-install.sh" -O "/tmp/fenics-install.sh" && bash "/tmp/fenics-install.sh"
    import dolfin

--2023-06-17 03:29:19--  https://fem-on-colab.github.io/releases/fenics-install.sh
Resolving fem-on-colab.github.io (fem-on-colab.github.io)... 185.199.108.153, 185.199.109.153, 185.199.110.153, ...
Connecting to fem-on-colab.github.io (fem-on-colab.github.io)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4156 (4.1K) [application/x-sh]
Saving to: ‘/tmp/fenics-install.sh’

/tmp/fenics-install 100%[===================>]   4.06K  --.-KB/s    in 0s      

2023-06-17 03:29:19 (35.1 MB/s) - ‘/tmp/fenics-install.sh’ saved [4156/4156]

+ INSTALL_PREFIX=/usr/local
++ echo /usr/local
++ awk -F/ '{print NF-1}'
+ INSTALL_PREFIX_DEPTH=2
+ PROJECT_NAME=fem-on-colab
+ SHARE_PREFIX=/usr/local/share/fem-on-colab
+ FENICS_INSTALLED=/usr/local/share/fem-on-colab/fenics.installed
+ [[ ! -f /usr/local/share/fem-on-colab/fenics.installed ]]
+ PYBIND11_INSTALL_SCRIPT_PATH=https://github.com/fem-on-colab/fem-on-colab.github.io/raw/6895377/releases/pybind11-install.s

In [ ]:
!pip install meshio==4.4.6

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.1/158.1 kB 3.9 MB/s eta 0:00:00


In [ ]:
!pip install pyDOE

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for pyDOE: filename=pyDOE-0.3.8-py3-none-any.whl size=18167 sha256=0110ad36131c7f047efd85d76035659183d0a5d9be8ba749bff69402d04f617e
  Stored in directory: /root/.cache/pip/wheels/ce/b6/d7/c6b64746dba6433c593e471e0ac3acf4f36040456d1d160d17
Successfully built pyDOE


In [ ]:
import meshio
import dolfin
import os
import matplotlib.pyplot as plt
import numpy as np
import time
import modelfunctions # manual import
import linearregression # manual import

from ast import Interactive
from dolfin import *
from modelfunctions import solve_poroelasticity
from linearregression import regression
from sklearn import linear_model
from pyDOE import lhs

## Parameters dataset creation

1. We specify a `range` around the baseline values of the considered lung parameter values under analysis:

Lung tissue constitutive model parameters:
*   $c$
*   $\beta$
*   $c_1$
*   $c_3$

Lung permeability:
*   $k$

Spring stiffness (chest-wall effect):
*   $K_s$


\
2. We use Latin Hypercube Sampling to obtain `sample_num` number of samples of the parameter space, bounded in the specified range.

In [ ]:
# We consider a range of 50% around the mean
range = 0/100

# Constitutive model parameters bounds (Bir. 2019 et al.)
C_bir2019    = [-range*356.7 + 356.7 , 356.7 +range*356.7] # parameter c
Beta_bir2019 = [-range*1.075 + 1.075 , 1.075 +range*1.075] # parameter beta
C1_bir2019   = [-range*278.2 + 278.2 , 278.2 +range*278.2] # parameter c1
C3_bir2019   = [-range*5.766 + 5.766 , 5.766 +range*5.766] # parameter c3

# Mechanical parameters bounds
per = [-range*10000 + 10000 , 10000 +range*10000]          # parameter k
KKresortee = [-range*0.08 + 0.08 , 0.08 +range*0.08]       # parameter Ks

# We use Latin Hypercube Sampling approach to generate space-filling training samples.
# Number of samples
sample_num = 2
# Bounds
lb = np.array([C_bir2019[0], Beta_bir2019[0], C1_bir2019[0], C3_bir2019[0], per[0], KKresortee[0]])
ub = np.array([C_bir2019[1], Beta_bir2019[1], C1_bir2019[1], C3_bir2019[1], per[1], KKresortee[1]])
# Generation of samples
X_data = (ub-lb)*lhs(6, samples=sample_num) + lb

# We create an output vector to store the posterior simulation results (Crs and R)
Y_data = np.zeros([sample_num, 2])
print(X_data, Y_data)

[[3.567e+02 1.075e+00 2.782e+02 5.766e+00 1.000e+04 8.000e-02]
 [3.567e+02 1.075e+00 2.782e+02 5.766e+00 1.000e+04 8.000e-02]] [[0. 0.]
 [0. 0.]]


## Performing the simulations

Next, for each sample of model parameters, the problem is solved using finite elements in FEniCS. To this end we call `solve_poroelasticity`, in which the following steps are performed:

*   Load the mesh and boundaryes
*   Build function space. We use Taylor-Hood element
*   Initialize solver
*   Time-stepping loop

At the end of each simulation, we obtain arrays of pressure, airflow, and volume waveforms, along with a time array.




In [ ]:
# Measuring time
start_time = time.time()

# Simulation loop (it will execute 'sample_num' number of cases)
i = 0

while i < len(X_data):

    ii = i

    C_bir2019    = X_data[i,0]
    Beta_bir2019 = X_data[i,1]
    C1_bir2019   = X_data[i,2]
    C3_bir2019   = X_data[i,3]
    per          = X_data[i,4]
    KKresortee   = X_data[i,5]

    models=['bir2019']
    for model in models:
        tiempos,Jacob,flux,presionestodas=solve_poroelasticity('TEST',model,'high',per,KKresortee,ii,C_bir2019,Beta_bir2019,C1_bir2019,C3_bir2019)

    i += 1

print("--- %s seconds ---" % (time.time() - start_time))

Calling FFC just-in-time (JIT) compiler, this may take some time.


Level 25:FFC:Calling FFC just-in-time (JIT) compiler, this may take some time.
INFO:FFC:Compiling element ffc_element_2269afe93cfdb70efbe7c05232b914e37e112d05

INFO:FFC:Compiler stage 1: Analyzing element(s)
INFO:FFC:--------------------------------------
INFO:FFC:  
INFO:FFC:Compiler stage 1 finished in 0.00466371 seconds.

INFO:FFC:Compiler stage 2: Computing intermediate representation
INFO:FFC:-------------------------------------------------------
INFO:FFC:  Computing representation of 1 elements
DEBUG:FFC:  Reusing element from cache
DEBUG:FFC:  Reusing element from cache
DEBUG:FFC:  Reusing element from cache
DEBUG:FFC:  Reusing element from cache
INFO:FFC:  Computing representation of 1 dofmaps
DEBUG:FFC:  Reusing element from cache
INFO:FFC:  Computing representation of 0 coordinate mappings
INFO:FFC:  Computing representation of integrals
INFO:FFC:  Computing representation of forms
INFO:FFC:  
INFO:FFC:Compiler stage 2 finished in 0.299077 seconds.

INFO:FFC:Compiler stage 3

Calling FFC just-in-time (JIT) compiler, this may take some time.


Level 25:FFC:Calling FFC just-in-time (JIT) compiler, this may take some time.
INFO:FFC:Compiling element ffc_element_b863195d8e0b681dc3527ba482c8cc38ad6593cf

INFO:FFC:Compiler stage 1: Analyzing element(s)
INFO:FFC:--------------------------------------
INFO:FFC:  
INFO:FFC:Compiler stage 1 finished in 0.0162098 seconds.

INFO:FFC:Compiler stage 2: Computing intermediate representation
INFO:FFC:-------------------------------------------------------
INFO:FFC:  Computing representation of 1 elements
DEBUG:FFC:  Reusing element from cache
DEBUG:FFC:  Reusing element from cache
DEBUG:FFC:  Reusing element from cache
INFO:FFC:  Computing representation of 1 dofmaps
DEBUG:FFC:  Reusing element from cache
INFO:FFC:  Computing representation of 0 coordinate mappings
INFO:FFC:  Computing representation of integrals
INFO:FFC:  Computing representation of forms
INFO:FFC:  
INFO:FFC:Compiler stage 2 finished in 0.0397577 seconds.

INFO:FFC:Compiler stage 3: Optimizing intermediate representatio

Calling FFC just-in-time (JIT) compiler, this may take some time.


...

Calling FFC just-in-time (JIT) compiler, this may take some time.


Level 25:FFC:Calling FFC just-in-time (JIT) compiler, this may take some time.
INFO:FFC:Compiling form ffc_form_ac4c55df7f7214bb27359c624eedd1c44c267da1

INFO:FFC:Compiler stage 1: Analyzing form(s)
INFO:FFC:-----------------------------------
DEBUG:FFC:  Preprocessing form using 'uflacs' representation family.
INFO:UFL:Adjusting missing element cell to tetrahedron.
INFO:FFC:  
INFO:FFC:  Geometric dimension:                 3
  Number of exterior_facet subdomains: 2
  Rank:                                0
  Arguments:                           '()'
  Number of coefficients:              1
  Coefficients:                        '[f_77]'
  Unique elements:                     'R0(?,?), Vector<3 x CG1(?,?)>'
  Unique sub elements:                 'R0(?,?), Vector<3 x CG1(?,?)>, CG1(?,?)'
  
INFO:FFC:  representation:    uflacs
INFO:FFC:  quadrature_rule:   auto --> default
INFO:FFC:  quadrature_degree: 3
INFO:FFC:  
INFO:FFC:Compiler stage 1 finished in 0.0389893 seconds.

INFO:FFC:Comp

Calling FFC just-in-time (JIT) compiler, this may take some time.


Level 25:FFC:Calling FFC just-in-time (JIT) compiler, this may take some time.
INFO:FFC:Compiling element ffc_element_ad70f7c6bfd34dec4d06cefe007cf369af7a3de2

INFO:FFC:Compiler stage 1: Analyzing element(s)
INFO:FFC:--------------------------------------
INFO:FFC:  
INFO:FFC:Compiler stage 1 finished in 0.00276804 seconds.

INFO:FFC:Compiler stage 2: Computing intermediate representation
INFO:FFC:-------------------------------------------------------
INFO:FFC:  Computing representation of 1 elements
DEBUG:FFC:  Reusing element from cache
DEBUG:FFC:  Reusing element from cache
DEBUG:FFC:  Reusing element from cache
INFO:FFC:  Computing representation of 1 dofmaps
DEBUG:FFC:  Reusing element from cache
INFO:FFC:  Computing representation of 0 coordinate mappings
INFO:FFC:  Computing representation of integrals
INFO:FFC:  Computing representation of forms
INFO:FFC:  
INFO:FFC:Compiler stage 2 finished in 0.012747 seconds.

INFO:FFC:Compiler stage 3: Optimizing intermediate representatio

PRESSURE,TIME,MODEL: 0.1182 0.02 bir2019
PRESSURE,TIME,MODEL: 0.1773 0.03 bir2019
PRESSURE,TIME,MODEL: 0.2364 0.04 bir2019
PRESSURE,TIME,MODEL: 0.2955 0.05 bir2019
PRESSURE,TIME,MODEL: 0.3546 0.06 bir2019
PRESSURE,TIME,MODEL: 0.4137 0.07 bir2019
PRESSURE,TIME,MODEL: 0.4728 0.08 bir2019
PRESSURE,TIME,MODEL: 0.5319 0.09 bir2019
PRESSURE,TIME,MODEL: 0.591 0.1 bir2019
PRESSURE,TIME,MODEL: 0.591 0.16 bir2019
PRESSURE,TIME,MODEL: 0.591 0.22000000000000003 bir2019
PRESSURE,TIME,MODEL: 0.591 0.28 bir2019
PRESSURE,TIME,MODEL: 0.591 0.34 bir2019
PRESSURE,TIME,MODEL: 0.591 0.4 bir2019
PRESSURE,TIME,MODEL: 0.591 0.4600000000000001 bir2019
PRESSURE,TIME,MODEL: 0.591 0.52 bir2019
PRESSURE,TIME,MODEL: 0.591 0.5800000000000001 bir2019
PRESSURE,TIME,MODEL: 0.591 0.64 bir2019
PRESSURE,TIME,MODEL: 0.591 0.7000000000000001 bir2019
PRESSURE,TIME,MODEL: 0.591 0.76 bir2019
PRESSURE,TIME,MODEL: 0.591 0.8200000000000001 bir2019
PRESSURE,TIME,MODEL: 0.591 0.88 bir2019
PRESSURE,TIME,MODEL: 0.591 0.94000000000000

## Obtaining respiratory-system compliance and airways resistance

Finally, from the ventilator signals and using the single compartment equation of motion, we obtain the respiratory-system compliance and airways resistance parameters via least-squares fitting by calling the `regression` function.

In [ ]:
# Simulation loop (it will execute 'sample_num' number of cases)
i = 0

while i < len(X_data):

    ii = i

    # Debe existir una carpeta llamada derecho donde estén las curvas del pulmón derecho. Idem para el izquierdo
    name='bir2019'

    ## We load the mechanical ventilation curves (considering both lungs)
    fluxes=fileflujos=np.load(name+str(ii)+'fluxes.npy')+np.load(name+str(ii)+'fluxes.npy')#[0:84]
    presionestodas=filepresiones=np.load(name+str(ii)+'presionestodas.npy')#+filepresionesi
    tiempos=filetiempos=np.load(name+str(ii)+'tiempos.npy')
    Jacob=filevolumenes=np.load(name+str(ii)+'volumenes.npy')+np.load(name+str(ii)+'volumenes.npy')#[0:84]
    Jacob=Jacob-Jacob[0]

    # Leemos npy generado de ambos pulmones
    maflujos=fluxes*60
    mapresiones=presionestodas
    matiempos=tiempos
    mavolumenes=Jacob

    # Agregamos punto 0
    maflujos=np.concatenate((np.array([0]),np.asarray(maflujos)))
    mapresiones=np.concatenate((np.array([0]),np.asarray(mapresiones)))
    matiempos=np.concatenate((np.array([0]),np.asarray(matiempos)))
    mavolumenes=np.concatenate((np.array([0]),np.asarray(mavolumenes)))


    regression(maflujos,mapresiones,matiempos,mavolumenes,name,Y_data,i)

    i += 1

bir2019
Resistance(R)= (3.8195726868217355, 2) cm H2O L/S, 
Compliance (Crs)= 93.62131911065362 ml/cm H2O
---------------
bir2019
Resistance(R)= (3.8195726868217355, 2) cm H2O L/S, 
Compliance (Crs)= 93.62131911065362 ml/cm H2O
---------------


## Display the dataset

Finally, we display the created dataset. From this, it can be observed that:

- `X_data` is an array that corresponds to the sampled lung parameters using the Latin hypercube design: each row a sample containing the values of the six parameters.

- `Y_data` is an array where a single row corresponds to the obtained respiratory-system compliance and airways resistance after simulating the corresponding sample of lung model parameters.

In [ ]:
print(X_data)
print(Y_data)

np.save('input_data.npy', X_data)
np.save('output_data.npy', Y_data)

[[3.567e+02 1.075e+00 2.782e+02 5.766e+00 1.000e+04 8.000e-02]
 [3.567e+02 1.075e+00 2.782e+02 5.766e+00 1.000e+04 8.000e-02]]
[[ 3.81957269 93.62131911]
 [ 3.81957269 93.62131911]]
